In [ ]:
import numpy as np
from sklearn import svm, metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow import keras
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array, array_to_img
from skimage.transform import resize, rescale
import matplotlib.pyplot as plt
from skimage import data, color
from PIL import Image
import cv2
from numpy import savez_compressed
from google.colab import files
from numpy import load
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
from google.colab import drive
drive.mount('/content/gdrive')

filename = '/content/gdrive/My Drive/Research2/np_data/xkmnist.npz'
x = np.load(filename)['arr_0']

filename = '/content/gdrive/My Drive/Research2/np_data/ykmnist.npz'
y = np.load(filename)['arr_0']

print(x.shape)
print(y.shape)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
(70000, 56, 56, 1)
(70000, 1)


In [ ]:
baseAccuracies = []
losses = []
balancedAccuracies = []
f1scores = []
precisions = []
recalls = []

kfold = KFold(n_splits=10, shuffle=True)
fold_no = 1
callback = EarlyStopping(monitor = 'val_loss', patience = 2, verbose = 1)

for train, test in kfold.split(x, y):
  print("_________________________________________________________________________________________________________________________________")
  print("Fold number: " + str(fold_no))
  fold_no = fold_no+1
  print()

  xtrain = x[train]
  xtest = x[test]
  
  model = tf.keras.applications.VGG19(include_top=True, weights=None, input_shape=(56, 56, 1), classes=10, classifier_activation="softmax",)
  model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(lr=0.0001), metrics=["sparse_categorical_accuracy"])
  model.fit(xtrain, y[train], verbose = 1, validation_data = (xtest, y[test]), epochs = 100, callbacks = [callback], batch_size = 32)

  test_loss, test_accuracy = model.evaluate(xtest, y[test])
  print("_________________________________________________________________________________________________________________________________")
  print("Test Accuracy:")
  print(test_accuracy)
  baseAccuracies.append(test_accuracy)
  print("_________________________________________________________________________________________________________________________________")
  print("Test Loss:")
  print(test_loss)
  losses.append(test_loss)
  print("_________________________________________________________________________________________________________________________________")
  preds = np.argmax(model.predict(xtest), axis=-1)
  preds = preds.reshape(len(xtest), 1)
  accs = []
  for cls in range(10):
    mask = (y[test] == cls)
    cls_acc = (preds == cls)[mask].mean() 
    accs.append(cls_acc)
  accs = np.mean(accs)
  print("Final balanced accuracy:")
  print(accs)
  balancedAccuracies.append(accs)
  print("_________________________________________________________________________________________________________________________________")
  f1scores.append(f1_score(y[test], preds, average = 'weighted'))
  precisions.append(precision_score(y[test], preds, average = 'weighted'))
  recalls.append(recall_score(y[test], preds, average = 'weighted'))
  print("*********************************************************************************************************************************")
  print()
  print()

_________________________________________________________________________________________________________________________________
Fold number: 1

Epoch 1/100
1969/1969 [==============================] - 68s 35ms/step - loss: 0.4704 - sparse_categorical_accuracy: 0.8430 - val_loss: 0.1282 - val_sparse_categorical_accuracy: 0.9611
Epoch 2/100
1969/1969 [==============================] - 68s 34ms/step - loss: 0.1050 - sparse_categorical_accuracy: 0.9699 - val_loss: 0.0804 - val_sparse_categorical_accuracy: 0.9767
Epoch 3/100
1969/1969 [==============================] - 68s 34ms/step - loss: 0.0684 - sparse_categorical_accuracy: 0.9814 - val_loss: 0.0708 - val_sparse_categorical_accuracy: 0.9814
Epoch 4/100
1969/1969 [==============================] - 68s 35ms/step - loss: 0.0522 - sparse_categorical_accuracy: 0.9859 - val_loss: 0.0560 - val_sparse_categorical_accuracy: 0.9847
Epoch 5/100
1969/1969 [==============================] - 68s 35ms/step - loss: 0.0393 - sparse_categorical_accurac

In [ ]:
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(balancedAccuracies)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {losses[i]} - class-wise Accuracy: {balancedAccuracies[i]}% Accuracy - {baseAccuracies[i]} - Precision: {precisions[i]} - Recall: {recalls[i]} - F1: {f1scores[i]}')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(balancedAccuracies)} (+- {np.std(balancedAccuracies)})')
print(f'> Loss: {np.mean(losses)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.03721766546368599 - class-wise Accuracy: 0.9904012290958979% Accuracy - 0.9904285669326782 - Precision: 0.9904589468543247 - Recall: 0.9904285714285714 - F1: 0.9904303789703008
------------------------------------------------------------------------
> Fold 2 - Loss: 0.06004111468791962 - class-wise Accuracy: 0.9845698420841253% Accuracy - 0.9845714569091797 - Precision: 0.9847629907513973 - Recall: 0.9845714285714285 - F1: 0.9845888215447187
------------------------------------------------------------------------
> Fold 3 - Loss: 0.062471117824316025 - class-wise Accuracy: 0.9844409659057929% Accuracy - 0.9844285845756531 - Precision: 0.9845948460502866 - Recall: 0.9844285714285714 - F1: 0.9844497015096757
------------------------------------------------------------------------
> Fold 4 - Loss: 0.066779315471